In [2]:
import pandas as pd
import numpy as np
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
import os


In [3]:
os.chdir('data')
log = pd.read_csv('log.tsv',delimiter="\t")
events = pd.read_csv('events.tsv', delimiter="\t")
users = pd.read_csv('users.tsv', delimiter="\t")
os.chdir('..')

In [4]:
print(log.shape)
log.head()

(1452089, 7)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN


In [5]:
print(events.shape)
events.head()

(158392, 11)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest
0,1,20,99.0,25,44.0,2015-03-08 18:00:00,宮城県,NaN,1500.0,5000.0,NaN
1,2,30,99.0,30,49.0,2015-03-01 15:00:00,富山県,NaN,1000.0,6500.0,NaN
2,3,22,99.0,24,39.0,2015-03-07 19:00:00,富山県,NaN,1000.0,6000.0,NaN
3,4,20,99.0,25,44.0,2015-03-08 18:00:00,新潟県,NaN,1000.0,6000.0,NaN
4,5,20,99.0,20,39.0,2015-03-14 19:00:00,新潟県,NaN,1000.0,6000.0,NaN


In [6]:
print(users.shape)
users.head()

(69667, 5)


,user_id,age,gender,prefecture,created_on
0,1,34,女性,静岡県,2012-06-26
1,2,31,男性,鳥取県,2012-07-12
2,3,32,男性,東京都,2012-07-15
3,4,30,男性,東京都,2012-07-17
4,5,29,女性,埼玉県,2012-07-17


http://surprise.readthedocs.io/en/stable/getting_started.html  
# Tutorialどおりにやってみる


In [7]:
import pandas as pd

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [8]:
table = log.loc[:,["user_id","event_id","action_type"]]
table.head()

,user_id,event_id,action_type
0,1,6261,3
1,1,127600,1
2,1,127600,1
3,1,125296,1
4,1,125128,1


In [9]:
reader = Reader(rating_scale=(1,3))
data = Dataset.load_from_df(table, reader)

In [10]:
train = data.build_full_trainset()

## チューニング

In [11]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV



param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, refit=True)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

KeyboardInterrupt: 

In [12]:
algo = SVD(n_epochs = 10, lr_all = 0.005, reg_all = 0.4)


In [13]:
algo.fit(train)

## 予測

In [14]:
os.chdir('data')
test = pd.read_csv('test.tsv',delimiter="\t")
os.chdir('..')

In [15]:
print(test.shape)
test.head()

(2886, 1)


,user_id
0,1
1,6
2,23
3,43
4,46


In [16]:
#tutorialに載ってる予測のしかた
uid = str(1)
iid = str(1)

pred = algo.predict(uid, iid, verbose=True)

#test全体に対して予測するのどうやるんだ・・・？
#for文？

user: 1          item: 1          r_ui = None   est = 1.59   {'was_impossible': False}


In [17]:
pred.est

1.594269359522729

In [18]:
user_list = test["user_id"]
event_list = events["event_id"]



In [19]:
#2017-9-24~2017-9-30のログを予想すればよいので，eventを絞る
events.loc[:,"event_date"] = pd.to_datetime(events["event_start_at"])

events = events.loc[events["event_date"] > pd.to_datetime("24/9/2017"),:]
events = events.reset_index(drop = True)
print(events.shape)
events.head()

event_list = events["event_id"]


(2422, 12)


In [27]:
predict_by_user = []#{(user_id,event_id):estimated_r}を格納
recommends = []
k= 20
i = 0

for user in user_list:
    predict_by_user = []
    i = i +1
    print("user_id:"+str(user))
    print("進捗:"+str(i*100/len(user_list))+"%") 
    for event in event_list:
        pred = algo.predict(user, event)
        
        predict = {"id":(user,event),"estimated_value":pred.est}
        predict_by_user.append(predict)
    
    recommends_by_user = sorted(predict_by_user, key=lambda x:x["estimated_value"], reverse=True)[:k]
    recommends.append(recommends_by_user)

user_id:1
進捗:0.03465003465003465%
user_id:6
進捗:0.0693000693000693%
user_id:23
進捗:0.10395010395010396%
user_id:43
進捗:0.1386001386001386%
user_id:46
進捗:0.17325017325017325%
user_id:48
進捗:0.2079002079002079%
user_id:76
進捗:0.24255024255024255%
user_id:79
進捗:0.2772002772002772%
user_id:104
進捗:0.31185031185031187%
user_id:139
進捗:0.3465003465003465%
user_id:152
進捗:0.38115038115038113%
user_id:198
進捗:0.4158004158004158%
user_id:206
進捗:0.45045045045045046%
user_id:233
進捗:0.4851004851004851%
user_id:242
進捗:0.5197505197505198%
user_id:247
進捗:0.5544005544005544%
user_id:263
進捗:0.589050589050589%
user_id:285
進捗:0.6237006237006237%
user_id:313
進捗:0.6583506583506583%
user_id:331
進捗:0.693000693000693%
user_id:333
進捗:0.7276507276507277%
user_id:346
進捗:0.7623007623007623%
user_id:387
進捗:0.796950796950797%
user_id:388
進捗:0.8316008316008316%
user_id:390
進捗:0.8662508662508662%
user_id:396
進捗:0.9009009009009009%
user_id:406
進捗:0.9355509355509356%
user_id:421
進捗:0.9702009702009702%
user_id:454
進捗:1.004851004

user_id:4705
進捗:8.142758142758142%
user_id:4714
進捗:8.177408177408177%
user_id:4759
進捗:8.212058212058212%
user_id:4762
進捗:8.246708246708247%
user_id:4776
進捗:8.281358281358282%
user_id:4850
進捗:8.316008316008316%
user_id:4864
進捗:8.350658350658351%
user_id:4898
進捗:8.385308385308385%
user_id:4951
進捗:8.41995841995842%
user_id:4956
進捗:8.454608454608454%
user_id:4966
進捗:8.48925848925849%
user_id:4976
進捗:8.523908523908524%
user_id:5003
進捗:8.558558558558559%
user_id:5007
進捗:8.593208593208594%
user_id:5009
進捗:8.627858627858627%
user_id:5014
進捗:8.662508662508662%
user_id:5059
進捗:8.697158697158697%
user_id:5086
進捗:8.731808731808732%
user_id:5123
進捗:8.766458766458767%
user_id:5128
進捗:8.801108801108802%
user_id:5144
進捗:8.835758835758837%
user_id:5158
進捗:8.87040887040887%
user_id:5168
進捗:8.905058905058905%
user_id:5169
進捗:8.93970893970894%
user_id:5181
進捗:8.974358974358974%
user_id:5182
進捗:9.00900900900901%
user_id:5208
進捗:9.043659043659044%
user_id:5274
進捗:9.07830907830908%
user_id:5275
進捗:9.11295911

user_id:9047
進捗:16.216216216216218%
user_id:9057
進捗:16.250866250866252%
user_id:9066
進捗:16.285516285516284%
user_id:9094
進捗:16.32016632016632%
user_id:9106
進捗:16.354816354816354%
user_id:9115
進捗:16.38946638946639%
user_id:9135
進捗:16.424116424116423%
user_id:9253
進捗:16.45876645876646%
user_id:9269
進捗:16.493416493416493%
user_id:9279
進捗:16.528066528066528%
user_id:9307
進捗:16.562716562716563%
user_id:9334
進捗:16.597366597366598%
user_id:9355
進捗:16.632016632016633%
user_id:9359
進捗:16.666666666666668%
user_id:9400
進捗:16.701316701316703%
user_id:9438
進捗:16.735966735966738%
user_id:9487
進捗:16.77061677061677%
user_id:9500
進捗:16.805266805266804%
user_id:9506
進捗:16.83991683991684%
user_id:9585
進捗:16.874566874566874%
user_id:9608
進捗:16.90921690921691%
user_id:9640
進捗:16.943866943866944%
user_id:9653
進捗:16.97851697851698%
user_id:9683
進捗:17.013167013167013%
user_id:9689
進捗:17.04781704781705%
user_id:9694
進捗:17.082467082467083%
user_id:9709
進捗:17.117117117117118%
user_id:9744
進捗:17.151767151767153%


user_id:13257
進捗:24.047124047124047%
user_id:13272
進捗:24.081774081774082%
user_id:13281
進捗:24.116424116424117%
user_id:13312
進捗:24.151074151074152%
user_id:13315
進捗:24.185724185724187%
user_id:13325
進捗:24.22037422037422%
user_id:13354
進捗:24.255024255024257%
user_id:13372
進捗:24.28967428967429%
user_id:13380
進捗:24.324324324324323%
user_id:13419
進捗:24.358974358974358%
user_id:13433
進捗:24.393624393624393%
user_id:13456
進捗:24.428274428274428%
user_id:13471
進捗:24.462924462924462%
user_id:13480
進捗:24.497574497574497%
user_id:13570
進捗:24.532224532224532%
user_id:13618
進捗:24.566874566874567%
user_id:13672
進捗:24.601524601524602%
user_id:13686
進捗:24.636174636174637%
user_id:13701
進捗:24.670824670824672%
user_id:13711
進捗:24.705474705474707%
user_id:13712
進捗:24.74012474012474%
user_id:13716
進捗:24.774774774774773%
user_id:13725
進捗:24.809424809424808%
user_id:13777
進捗:24.844074844074843%
user_id:13809
進捗:24.878724878724878%
user_id:13830
進捗:24.913374913374913%
user_id:13833
進捗:24.948024948024948%
user

user_id:18191
進捗:31.947331947331946%
user_id:18278
進捗:31.98198198198198%
user_id:18283
進捗:32.016632016632016%
user_id:18432
進捗:32.05128205128205%
user_id:18444
進捗:32.085932085932086%
user_id:18450
進捗:32.12058212058212%
user_id:18460
進捗:32.155232155232156%
user_id:18464
進捗:32.18988218988219%
user_id:18473
進捗:32.224532224532226%
user_id:18489
進捗:32.25918225918226%
user_id:18498
進捗:32.293832293832295%
user_id:18533
進捗:32.32848232848233%
user_id:18537
進捗:32.363132363132365%
user_id:18583
進捗:32.3977823977824%
user_id:18584
進捗:32.432432432432435%
user_id:18611
進捗:32.46708246708247%
user_id:18622
進捗:32.501732501732505%
user_id:18675
進捗:32.53638253638254%
user_id:18705
進捗:32.57103257103257%
user_id:18732
進捗:32.6056826056826%
user_id:18744
進捗:32.64033264033264%
user_id:18756
進捗:32.67498267498267%
user_id:18778
進捗:32.70963270963271%
user_id:18787
進捗:32.74428274428274%
user_id:18795
進捗:32.77893277893278%
user_id:18807
進捗:32.81358281358281%
user_id:18813
進捗:32.84823284823285%
user_id:18842
進捗:32.8

...

user_id:51155
進捗:63.89466389466389%
user_id:51189
進捗:63.92931392931393%
user_id:51219
進捗:63.96396396396396%
user_id:51319
進捗:63.998613998614%
user_id:51446
進捗:64.03326403326403%
user_id:51453
進捗:64.06791406791407%
user_id:51458
進捗:64.1025641025641%
user_id:51484
進捗:64.13721413721414%
user_id:51487
進捗:64.17186417186417%
user_id:51498
進捗:64.2065142065142%
user_id:51510
進捗:64.24116424116424%
user_id:51524
進捗:64.27581427581427%
user_id:51535
進捗:64.31046431046431%
user_id:51632
進捗:64.34511434511434%
user_id:51692
進捗:64.37976437976438%
user_id:51784
進捗:64.41441441441441%
user_id:51789
進捗:64.44906444906445%
user_id:51820
進捗:64.48371448371448%
user_id:51846
進捗:64.51836451836452%
user_id:51856
進捗:64.55301455301455%
user_id:51908
進捗:64.58766458766459%
user_id:51947
進捗:64.62231462231462%
user_id:51996
進捗:64.65696465696466%
user_id:52071
進捗:64.69161469161469%
user_id:52149
進捗:64.72626472626473%
user_id:52165
進捗:64.76091476091476%
user_id:52213
進捗:64.7955647955648%
user_id:52255
進捗:64.8302148302148

user_id:59859
進捗:71.82952182952182%
user_id:59863
進捗:71.86417186417187%
user_id:59877
進捗:71.8988218988219%
user_id:59883
進捗:71.93347193347194%
user_id:59913
進捗:71.96812196812196%
user_id:59928
進捗:72.002772002772%
user_id:59946
進捗:72.03742203742203%
user_id:59953
進捗:72.07207207207207%
user_id:59958
進捗:72.1067221067221%
user_id:59975
進捗:72.14137214137214%
user_id:60006
進捗:72.17602217602217%
user_id:60010
進捗:72.21067221067221%
user_id:60012
進捗:72.24532224532224%
user_id:60035
進捗:72.27997227997228%
user_id:60057
進捗:72.31462231462231%
user_id:60060
進捗:72.34927234927235%
user_id:60084
進捗:72.38392238392238%
user_id:60145
進捗:72.41857241857242%
user_id:60159
進捗:72.45322245322245%
user_id:60180
進捗:72.4878724878725%
user_id:60213
進捗:72.52252252252252%
user_id:60244
進捗:72.55717255717256%
user_id:60277
進捗:72.59182259182259%
user_id:60314
進捗:72.62647262647263%
user_id:60327
進捗:72.66112266112266%
user_id:60361
進捗:72.69577269577269%
user_id:60431
進捗:72.73042273042273%
user_id:60448
進捗:72.7650727650727

user_id:66331
進捗:79.83367983367984%
user_id:66338
進捗:79.86832986832987%
user_id:66346
進捗:79.90297990297991%
user_id:66372
進捗:79.93762993762994%
user_id:66393
進捗:79.97227997227998%
user_id:66394
進捗:80.00693000693%
user_id:66395
進捗:80.04158004158005%
user_id:66412
進捗:80.07623007623008%
user_id:66431
進捗:80.11088011088012%
user_id:66448
進捗:80.14553014553015%
user_id:66498
進捗:80.18018018018019%
user_id:66531
進捗:80.21483021483022%
user_id:66578
進捗:80.24948024948024%
user_id:66583
進捗:80.28413028413028%
user_id:66635
進捗:80.31878031878031%
user_id:66640
進捗:80.35343035343035%
user_id:66666
進捗:80.38808038808038%
user_id:66683
進捗:80.42273042273042%
user_id:66699
進捗:80.45738045738045%
user_id:66708
進捗:80.4920304920305%
user_id:66735
進捗:80.52668052668052%
user_id:66743
進捗:80.56133056133056%
user_id:66748
進捗:80.59598059598059%
user_id:66751
進捗:80.63063063063063%
user_id:66769
進捗:80.66528066528066%
user_id:66782
進捗:80.6999306999307%
user_id:66796
進捗:80.73458073458073%
user_id:66838
進捗:80.7692307692307

user_id:69577
進捗:87.87248787248787%
user_id:69578
進捗:87.90713790713791%
user_id:69580
進捗:87.94178794178794%
user_id:69582
進捗:87.97643797643798%
user_id:69584
進捗:88.011088011088%
user_id:69588
進捗:88.04573804573805%
user_id:69598
進捗:88.08038808038808%
user_id:69599
進捗:88.11503811503812%
user_id:69603
進捗:88.14968814968815%
user_id:69605
進捗:88.18433818433819%
user_id:69607
進捗:88.21898821898822%
user_id:69616
進捗:88.25363825363826%
user_id:69620
進捗:88.28828828828829%
user_id:69621
進捗:88.32293832293833%
user_id:69646
進捗:88.35758835758836%
user_id:69647
進捗:88.3922383922384%
user_id:69649
進捗:88.42688842688842%
user_id:69654
進捗:88.46153846153847%
user_id:69660
進捗:88.4961884961885%
user_id:69663
進捗:88.53083853083854%
user_id:69664
進捗:88.56548856548856%
user_id:69668
進捗:88.6001386001386%
user_id:69671
進捗:88.63478863478863%
user_id:69672
進捗:88.66943866943868%
user_id:69673
進捗:88.7040887040887%
user_id:69674
進捗:88.73873873873873%
user_id:69675
進捗:88.77338877338877%
user_id:69676
進捗:88.8080388080388%

user_id:69883
進捗:95.94594594594595%
user_id:69884
進捗:95.98059598059598%
user_id:69885
進捗:96.01524601524602%
user_id:69886
進捗:96.04989604989605%
user_id:69887
進捗:96.08454608454609%
user_id:69888
進捗:96.11919611919612%
user_id:69889
進捗:96.15384615384616%
user_id:69890
進捗:96.18849618849619%
user_id:69891
進捗:96.22314622314623%
user_id:69892
進捗:96.25779625779626%
user_id:69893
進捗:96.29244629244629%
user_id:69894
進捗:96.32709632709633%
user_id:69895
進捗:96.36174636174636%
user_id:69896
進捗:96.3963963963964%
user_id:69897
進捗:96.43104643104643%
user_id:69898
進捗:96.46569646569647%
user_id:69899
進捗:96.5003465003465%
user_id:69900
進捗:96.53499653499654%
user_id:69901
進捗:96.56964656964657%
user_id:69902
進捗:96.60429660429661%
user_id:69903
進捗:96.63894663894663%
user_id:69904
進捗:96.67359667359668%
user_id:69905
進捗:96.7082467082467%
user_id:69906
進捗:96.74289674289675%
user_id:69907
進捗:96.77754677754677%
user_id:69908
進捗:96.81219681219682%
user_id:69909
進捗:96.84684684684684%
user_id:69910
進捗:96.88149688149

In [28]:
recommends[1]

[{'estimated_value': 2.2118902472086828, 'id': (6, 152511)},
 {'estimated_value': 2.1882910939581826, 'id': (6, 146349)},
 {'estimated_value': 2.1318924307027305, 'id': (6, 153977)},
 {'estimated_value': 2.1041212166160186, 'id': (6, 156297)},
 {'estimated_value': 2.1010542364788614, 'id': (6, 152120)},
 {'estimated_value': 2.098824439459012, 'id': (6, 155206)},
 {'estimated_value': 2.07958138515677, 'id': (6, 148906)},
 {'estimated_value': 2.0786592412727574, 'id': (6, 157488)},
 {'estimated_value': 2.0704958494473114, 'id': (6, 146305)},
 {'estimated_value': 2.0698265779348195, 'id': (6, 153526)},
 {'estimated_value': 2.0687500100489213, 'id': (6, 150578)},
 {'estimated_value': 2.068389747839008, 'id': (6, 157487)},
 {'estimated_value': 2.0658178306173345, 'id': (6, 156258)},
 {'estimated_value': 2.064533065664709, 'id': (6, 150532)},
 {'estimated_value': 2.0642342052671014, 'id': (6, 151258)},
 {'estimated_value': 2.059057499216154, 'id': (6, 158236)},
 {'estimated_value': 2.0571117

In [29]:
f = open('SVD_recommend.txt', 'w')
for x in recommends:
    f.write(str(x) + "\n")
f.close()

In [30]:
df = pd.DataFrame(recommends)
df = pd.DataFrame(df.values.flatten())
df = df.rename(columns={0: "result"})
df.head()

,result
0,"{'id': (1, 152511), 'estimated_value': 1.58718..."
1,"{'id': (1, 146349), 'estimated_value': 1.56400..."
2,"{'id': (1, 153977), 'estimated_value': 1.50591..."
3,"{'id': (1, 156297), 'estimated_value': 1.48049..."
4,"{'id': (1, 152120), 'estimated_value': 1.47649..."


In [31]:
df.loc[:,"user_id"] = df["result"].apply(lambda x:x["id"][0])
df.loc[:,"event_id"] = df["result"].apply(lambda x:x["id"][1])
df.loc[:,"estimated_value"] = df["result"].apply(lambda x:x["estimated_value"])
#df.loc[:,"rank"] = [20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1] * len(user_list) 
df.loc[:,"rank"] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] * len(user_list) 

In [32]:
df.head(50)

,result,user_id,event_id,estimated_value,rank
0,"{'id': (1, 152511), 'estimated_value': 1.58718...",1,152511,1.587187,1
1,"{'id': (1, 146349), 'estimated_value': 1.56400...",1,146349,1.564009,2
2,"{'id': (1, 153977), 'estimated_value': 1.50591...",1,153977,1.505912,3
3,"{'id': (1, 156297), 'estimated_value': 1.48049...",1,156297,1.480492,4
4,"{'id': (1, 152120), 'estimated_value': 1.47649...",1,152120,1.476493,5
5,"{'id': (1, 155206), 'estimated_value': 1.46866...",1,155206,1.468664,6
6,"{'id': (1, 148906), 'estimated_value': 1.45139...",1,148906,1.451397,7
7,"{'id': (1, 157488), 'estimated_value': 1.45074...",1,157488,1.450747,8
8,"{'id': (1, 153526), 'estimated_value': 1.44634...",1,153526,1.446346,9
9,"{'id': (1, 157487), 'estimated_value': 1.44230...",1,157487,1.442302,10


In [33]:
submition = df[["user_id","event_id","rank"]]
print(submition.shape)
submition.head()

(57720, 3)


,user_id,event_id,rank
0,1,152511,1
1,1,146349,2
2,1,153977,3
3,1,156297,4
4,1,152120,5


In [34]:
submition.to_csv("sub.tsv",header=None,index=None,sep = "\t")